In [7]:
import pandas as pd
import os

# Load Excel file
file_path = "/Users/biekeleenknegt/Desktop/Supply Chain Management/SCM_GW/Real Demand.xlsx"
sheet_name = "Real Demand"

# Load data
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Rename 'season' to 'year'
df = df.rename(columns={'season': 'year'})

# Sort by year
df = df.sort_values(by='year')

# Determine how many years are available per product-size-country
year_counts = df.groupby(['product_id', 'size', 'channel_id'])['year'].nunique().reset_index()
year_counts = year_counts.rename(columns={'year': 'max_years'})

# Merge with full dataset so each row knows how many years are available
df = df.merge(year_counts, on=['product_id', 'size', 'channel_id'], how='left')

# Create output folder
output_folder = "/Users/biekeleenknegt/Documents/GitHub/SCM_GW/Forecasts_By_Window"
os.makedirs(output_folder, exist_ok=True)

# Dictionary to collect forecasts per window
all_window_dfs = dict()

# Loop over all product-size-channel combinations
for _, row in year_counts.iterrows():
    product = row['product_id']
    size = row['size']
    channel = row['channel_id']
    max_years = row['max_years']
    
    sub_df = df[(df['product_id'] == product) &
                (df['size'] == size) &
                (df['channel_id'] == channel)].copy()
    
    sub_df = sub_df.sort_values(by='year')
    
    for window in range(1, max_years + 1):
        sub_df[f'ma_{window}'] = sub_df['real demand'].rolling(window=window, min_periods=1).mean()
        forecast_row = sub_df.tail(1).copy()
        forecast_row['window'] = window
        forecast_row['forecast'] = forecast_row[f'ma_{window}']
        
        # Add relevant columns
        forecast_row = forecast_row[['product_id', 'size', 'channel_id', 'year', 'forecast']]
        
        if window not in all_window_dfs:
            all_window_dfs[window] = []
        all_window_dfs[window].append(forecast_row)

# Write each window's forecasts to a separate Excel file
for window, dfs in all_window_dfs.items():
    result_df = pd.concat(dfs, ignore_index=True)
    output_path = os.path.join(output_folder, f"Forecast_Window_{window}.xlsx")
    result_df.to_excel(output_path, index=False)

print(f"Forecasts per product-size-channel written to: {output_folder}")

Forecasts per product-size-channel written to: /Users/biekeleenknegt/Documents/GitHub/SCM_GW/Forecasts_By_Window
